# Using the MANN Package to convert and prune an existing TensorFlow model

In this notebook, we utilize the MANN package on an existing TensorFlow model to convert existing layers to MANN layers and then prune the model.

In [ ]:
# Load the MANN package and TensorFlow
import tensorflow as tf
import beyondml.tflow as mann

In [ ]:
# Load the data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train/255
x_test = x_test/255

# Load the model to be used
vgg16 = tf.keras.applications.VGG16(
    include_top = False,             # Don't include the top layers
    weights = 'imagenet',            # Load the imagenet weights
    input_shape = x_train.shape[1:]  # Input shape is the shape of the images
)

## Create the model to be trained

In the following cell, we create the model using the existing VGG model fed into fully-connected layers.

In [ ]:
# Build the model using VGG16 and a few layers on top of it
model = tf.keras.models.Sequential()
model.add(vgg16)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation = 'relu'))
model.add(tf.keras.layers.Dense(512, activation = 'relu'))
model.add(tf.keras.layers.Dense(512, activation = 'relu'))
model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

# Compile the model
model.compile(
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)

# Present model summary
model.summary()

## Convert the model and perform initial pruning

In the following cell, we convert the model and perform initial pruning of the model to 40%.

In [ ]:
# Use the add_layer_masks function to add masking layers to the model
converted_model = mann.utils.add_layer_masks(model)

# Compile the model
converted_model.compile(
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)

# Mask the model using magnitude as the metric
converted_model = mann.utils.mask_model(
    converted_model,
    40,
    method = 'magnitude'
)

# Recompile the model for the weights to take effect
converted_model.compile(
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)

# Present the model summary
converted_model.summary()

## Train and further prune the model

In this cell, we create the ActiveSparsification callback and train the model using that callback to prune the model as the model improves in performance.

In [ ]:
# Create the sparsification callback object
callback = mann.utils.ActiveSparsification(
    performance_cutoff = 0.75,    # The accuracy score the model needs to achieve
    starting_sparsification = 40, # Starting sparsification
    sparsification_rate = 5       # Sparsification increase every time the model achieves performance cutoff
)

# Fit the model
model.fit(
    x_train,
    y_train,
    epochs = 1000,
    callbacks = [callback],
    validation_split = 0.2,
    batch_size = 256
)

## Convert the model back to remove masking layers

In the following cell, we remove the layer masks created for training, while completely preserving performance.

In [ ]:
# Convert the model back
model = mann.utils.remove_layer_masks(model)

# Present the model
model.summary()

## Report accuracy and save model

In [ ]:
# Get the predictions on test data
preds = model.predict(x_test).argmax(axis = 1)

# Print the accuracy
print(f'Model Accuracy: {(preds.flatten() == y_test.flatten()).sum().astype(int)/y_test.flatten().shape[0]}')

# Save the model
model.save('cifar_vgg16.h5')